In [1]:
# Used for processing raw data files after fetching from source
# Preprocess is performed for all data files in the directory, regardless of the CLASS


In [2]:
#### FOR ACADEMIC PROJECT WORK
#check if lib install or not, if not, install

# !pip install wfdb
# !pip install lightgbm
# !pip install PyWavelets
# !pip install biosppy
# !pip install torch
# !pip install shap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 3.6 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.5 MB/s eta 0:00:00:00:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 33.4 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 38.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 3.5 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 43.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 22.0 MB/s eta 0:00:00:00:0100:01

[notice] A new rel

In [3]:
import re
from datetime import datetime
import shutil
import os
import pandas as pd
from glob import glob
import wfdb
#key parameters:
split_type = os.path.sep #newly added. Long. 23.Mar.24
dx_dict = {
    '426783006': 'SNR', # Normal sinus rhythm
    '164889003': 'AF', # Atrial fibrillation
    '270492004': 'IAVB', # First-degree atrioventricular block
    '164909002': 'LBBB', # Left bundle branch block
    '713427006': 'RBBB', # Complete right bundle branch block
    '59118001': 'RBBB', # Right bundle branch block
    '284470004': 'PAC', # Premature atrial contraction
    '63593006': 'PAC', # Supraventricular premature beats
    '164884008': 'PVC', # Ventricular ectopics
    '429622005': 'STD', # ST-segment depression
    '164931005': 'STE', # ST-segment elevation
}
classes = ['SNR', 'AF', 'IAVB', 'LBBB', 'RBBB', 'PAC', 'PVC', 'STD', 'STE']

In [3]:
#1. FUNCTION READ, AND MODIFY FILES
#Read data from each INDIVIDUAL file, modify it in the right format
# and save it back

def process_file(file_name):
    with open(file_name, 'r') as file:
        main_content = file.read()
    # main_content = re.search(r'<code>(.*?)</code>', main_content, re.DOTALL) #no need to get part of the file
    file.close()
    if main_content:
        # lines = main_content.group(1).split('\n')
        lines = main_content.split('\n')
        now = datetime.now()
        dt_string = now.strftime("%d-%b-%Y %H:%M:%S")
        lines[0] = lines[0] + ' ' + dt_string #append date time to first line
        for i in range(1, 13):
            #get location of '.mat' in the line
            mat_loc = lines[i].find('.mat')+4 #location id at beginning of '.mat' -> +4 till the end
            # print(mat_loc)
            lines[i] = lines[i][:mat_loc+3] + lines[i][mat_loc+5:] #remove 'x1'
            lines[i] = lines[i][:mat_loc+11] + lines[i][mat_loc+16:] #remove '.0(0)'
        for i in range(13, len(lines)):
            lines[i] = lines[i].replace('# ', '#') #replace "#" with "# " in the remaining lines
        new_content = '\n'.join(lines)
        # new_file_name = file_name.replace('.hea', '_new.hea') #no need to change the file name
        #overwrite the original file
        with open(file_name, 'w') as file:
            file.write(new_content)
            file.close()
        # print('New file created:', file_name)
    else:
        print('No match')

In [4]:
#2. FUNCTION MOVE FILES
#Within a folder, move all files to from subfolders to main folder
def copy_files(folder, N):
    files = os.listdir(folder)
    #if N = 0, loop all files
    if N == 0:
        N = len(files)

    for i in range(N):
        file_name = os.path.join(folder, files[i])
        new_folder = os.path.dirname(folder) #copy to upper folder
        # new_folder = os.path.join(os.path.dirname(folder), 'main-data')
        shutil.copy(file_name, new_folder)
        # print('Copied:', file_name)
    print('Done, ', N, 'files copied in', folder)

In [5]:
#3. FUNCTION MOVE AND PROCESS FILES
# Call copy_files to move files from subfolders to main folder
# Call process_file to make necessary changes for each files in the main folder
#Modification is done for all files, regardless of the class

def main_raw_files_process(data_folder, N):
    #STEP 1: Copy N files from each sub-folder to main folder

    #make a list of sub-folders within the main data folder
    folders = os.listdir(data_folder)
    #loop through the folders and copy N files from each sub-folder
    for folder in folders:
        folder_name = os.path.join(data_folder, folder)
        if os.path.isdir(folder_name): #filter dirs only
            copy_files(folder_name, N)
    print('Done copying files from subfolders to main folder:', data_folder)

    #STEP 2: Process each file in the main folder
    #loop through the files in main folder and process each file
    files = os.listdir(data_folder)
    #make full path to each file
    files = [os.path.join(data_folder, file) for file in files]
    for file in files:
        if file.endswith('.hea'):
            process_file(file)
    print('Done processing folders:', data_folder)


In [6]:
#4. function to move files .hea and .mat from one folder to another.
#Files name are extracted from 1st column of df

def move_files(df, source_folder, dest_folder):
    for index, row in df.iterrows():
        file_name = row['File']
        #append '.hea' and '.mat' to the file name, and move the files, one by one
        for ext in ['.hea', '.mat']:
            source_file = os.path.join(source_folder, file_name + ext)
            dest_file = os.path.join(dest_folder, file_name + ext)
            shutil.move(source_file, dest_file)
    print('Done moving files')

In [7]:
#5. Move files to corresponding folders based on the class
def conditional_move(data_dir,des_dir,split_type,dx_dict,classes):
    #phase 1: read and create labels for each class
    recordpaths = glob(os.path.join(data_dir, '*.hea'))
    results = []
    for recordpath in recordpaths:
        patient_id = recordpath.split(split_type)[-1][:-4]
        _, meta_data = wfdb.rdsamp(recordpath[:-4]) 
        dx = meta_data['comments'][2]
        dx = dx[4:] if dx.startswith('Dx: ') else ''
        results.append([patient_id, dx])
    df = pd.DataFrame(data=results, columns=['patient_id', 'dx'])
    # print('Check 1')
    # #view df
    # print(df.head())

    #phase 2: create labels for each class
    results = []
    for _, row in df.iterrows():
        patient_id = row['patient_id']
        dxs = [dx_dict.get(code, '') for code in row['dx'].split(',')]
        # labels = [0] * 9
        labels = [0] * len(classes) #Modified to handle additional classes. Long. 21.Apr.24
        for idx, label in enumerate(classes):
            if label in dxs:
                labels[idx] = 1
        results.append([patient_id] + labels)
    df = pd.DataFrame(data=results, columns=['patient_id'] + classes)
    # print('Check 2')
    # #view df
    # print(df.head())

    #only keep records exist in classes list. Long. 05.May.24
    df['keep'] = df[classes].sum(axis=1) #sum of all classes
    df = df[df['keep'] > 0] #at least one class is 1
    
    #print number of keep records, number of results, percentage of keep records vs results
    print('Number of keep records:', len(df), 
            '\nNumber of results:', len(results), 
            '\nPercentage of keep records:', len(df)/len(results)*100)       
    
    #after filtering those keep records,
    #move those to new destination folder:
    # des_dir = 'data/test_dataset'
    if os.path.exists(des_dir):
        for index, row in df.iterrows():
            file_name = row['patient_id'] #extract file name from 1st column
            # print('File name:', file_name)
            #append '.hea' and '.mat' to the file name, and move the files, one by one
            for ext in ['.hea', '.mat']:
                source_file = os.path.join(data_dir, file_name + ext)
                dest_file = os.path.join(des_dir, file_name + ext)
                shutil.move(source_file, dest_file)
            print('Done moving files')
    else:
        print('Destination folder does not exist:', des_dir)

In [ ]:
#cpsc_2018_extra
#modify files from raw downloaded data, select files based on the class, move files to train_dataset or test_dataset
source_folder = 'data\cpsc_2018_extra'
des_folder = 'data\op_09_classes\\test_dataset'
main_raw_files_process(source_folder, 0)
conditional_move(source_folder,des_folder,split_type,dx_dict,classes)

In [ ]:
#cpsc_2018
#modify files from raw downloaded data, select files based on the class, move files to train_dataset or test_dataset
source_folder = 'data\cpsc_2018'
des_folder = 'data\op_09_classes\\test_dataset'
main_raw_files_process(source_folder, 0)
conditional_move(source_folder,des_folder,split_type,dx_dict,classes)

In [ ]:
#ptb-xl
#modify files from raw downloaded data, select files based on the class, move files to train_dataset or test_dataset
source_folder = 'data\ptb-xl'
des_folder = 'data\op_09_classes\\train_dataset'
main_raw_files_process(source_folder, 0)
conditional_move(source_folder,des_folder,split_type,dx_dict,classes)

In [4]:
#Handle zip files, after uploading from local to codespaces

#loop through each zip file in data folder, and unzip it
zip_files = glob('data/*.zip')
for zip_file in zip_files:
    print('Unzipping:', zip_file)
    shutil.unpack_archive(zip_file, 'data')
    print('Unzipped:', zip_file)
    
#create folder archived within data folder, and move all zip files to this folder, to backup
if not os.path.exists('data/archived'):
    os.makedirs('data/archived')
    print('Folder created: data/archived')
for zip_file in zip_files:
    shutil.move(zip_file, 'data/archived')
    print('Moved:', zip_file)

Unzipping: data/cpsc_processed.zip
Unzipped: data/cpsc_processed.zip
Unzipping: data/op_09_classes.zip
Unzipped: data/op_09_classes.zip
Unzipping: data/op_08_classes.zip
Unzipped: data/op_08_classes.zip
Folder created: data/archived
Moved: data/cpsc_processed.zip
Moved: data/op_09_classes.zip
Moved: data/op_08_classes.zip


In [5]:
#Verify results
#check number of files in each subfolder within data folder
folders = os.listdir('data')

for folder in folders:
    #if not folder name archived, then proceed
    if folder == 'op_08_classes' or folder == 'op_09_classes':     
        folder_name = os.path.join('data', folder)    
        subfolders = os.listdir(folder_name)
        for subfolder in subfolders:
            subfolder_name = os.path.join(folder_name, subfolder)    
            #print number of files in each subfolder
            files = os.listdir(subfolder_name)
            print('Number of files in', subfolder_name, ':', len(files))
    #else if
    elif folder != 'archived' and folder != '.gitkeep':
        folder_name = os.path.join('data', folder)    
        #print number of files in each subfolder
        files = os.listdir(folder_name)
        print('Number of files in', folder_name, ':', len(files))   




Number of files in data/op_08_classes/train_dataset : 55649
Number of files in data/op_08_classes/test_dataset : 9248
Number of files in data/op_09_classes/train_dataset : 40674
Number of files in data/op_09_classes/test_dataset : 14976
Number of files in data/cpsc_processed : 13756
